In [36]:
# Import Libraries
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

In [37]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])
print('reviews:')
print(reviews)
print('sentiment:', sentiment)

reviews:
['nice food', 'amazing restaurant', 'too good', 'just loved it!', 'will go again', 'horrible food', 'never go there', 'poor service', 'poor quality', 'needs improvement']
sentiment: [1 1 1 1 1 0 0 0 0 0]


In [38]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/one_hot
# tf.keras.processing.text.on_hot(input_text, n) give the maximum number [0, n=30].
# Postive 
print('Positve:')
print('one_hot("nice food", 30):', one_hot("nice food", 30))
print('one_hot("amazing restaurant", 30):', one_hot("amazing restaurant", 30))
print('one_hot("too good", 30):', one_hot("too good", 30))
print('one_hot("just loved it!", 30):', one_hot("just loved it!", 30))
print('one_hot("will go again", 30):', one_hot("will go again", 30))
# Negative
print()
print('Negative:')
print('one_hot("horrible food", 30):', one_hot("horrible food", 30))
print('one_hot("never go there", 30):', one_hot("never go there", 30))
print('one_hot("poor service", 30):', one_hot("poor service", 30))
print('one_hot("poor quality", 30):', one_hot("poor quality", 30))
print('one_hot("needs improvement", 30):', one_hot("needs improvement", 30))

Positve:
one_hot("nice food", 30): [7, 21]
one_hot("amazing restaurant", 30): [15, 10]
one_hot("too good", 30): [22, 6]
one_hot("just loved it!", 30): [10, 21, 7]
one_hot("will go again", 30): [21, 29, 16]

Negative:
one_hot("horrible food", 30): [21, 21]
one_hot("never go there", 30): [29, 29, 22]
one_hot("poor service", 30): [4, 15]
one_hot("poor quality", 30): [4, 26]
one_hot("needs improvement", 30): [26, 20]


In [39]:
# Convert all the reivews into one_hot ecoding.
vocab_size = 30
encoded_reviews = [one_hot(d, vocab_size) for d in reviews]
print('encoded_reviews:')
print(encoded_reviews)

encoded_reviews:
[[7, 21], [15, 10], [22, 6], [10, 21, 7], [21, 29, 16], [21, 21], [29, 29, 22], [4, 15], [4, 26], [26, 20]]


In [40]:
# We have the input sequences. Each sequences contains words.
# What is the max word count of each input sequence?
max_length = 3
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print('padded_reviews:')
print(padded_reviews)

padded_reviews:
[[ 7 21  0]
 [15 10  0]
 [22  6  0]
 [10 21  7]
 [21 29 16]
 [21 21  0]
 [29 29 22]
 [ 4 15  0]
 [ 4 26  0]
 [26 20  0]]


In [41]:
# Specify the embedded vector size.
# vocab_size = 30
# max_length = 3
embeded_vector_size = 4

model = Sequential()
model.add(Embedding(vocab_size, embeded_vector_size, input_length=max_length,name="embedding"))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [42]:
# input sequences in encoding forms: X and 
# output target: y
X = padded_reviews
y = sentiment
print('X:')
print(X)
print('y:', y)

X:
[[ 7 21  0]
 [15 10  0]
 [22  6  0]
 [10 21  7]
 [21 29 16]
 [21 21  0]
 [29 29 22]
 [ 4 15  0]
 [ 4 26  0]
 [26 20  0]]
y: [1 1 1 1 1 0 0 0 0 0]


In [43]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 4)              120       
                                                                 
 flatten_7 (Flatten)         (None, 12)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 13        
                                                                 
Total params: 133 (532.00 Byte)
Trainable params: 133 (532.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [44]:
model.fit(X, y, epochs=50, verbose=0)

In [50]:
# evaluate the model
loss, accuracy = model.evaluate(X, y)
print('accuracy:')
print(accuracy)

1/1 [==============================] - 0s 20ms/step - loss: 0.6223 - accuracy: 1.0000
accuracy:
1.0


In [51]:
weights = model.get_layer('embedding').get_weights()[0]
print('len(weights):', len(weights))
print('weights:')
print(weights)

len(weights): 30
weights:
[[-0.06965557  0.00367606  0.08237346  0.05216464]
 [-0.02823919 -0.01955552  0.01174606 -0.03703552]
 [ 0.03013924 -0.00497038  0.0425275   0.02551112]
 [-0.02646829 -0.02060129 -0.04602636  0.04875183]
 [ 0.07662225  0.07200582  0.07823209 -0.0928142 ]
 [-0.04724219 -0.01749627 -0.03020895 -0.03521077]
 [ 0.06728394  0.00773586 -0.02382978  0.05918522]
 [ 0.01910876 -0.0501948  -0.08391881  0.06923967]
 [ 0.04591669  0.02584973  0.03441927 -0.04224567]
 [-0.00800632  0.00128511 -0.02568655 -0.01523328]
 [ 0.07713357 -0.0597582  -0.0197264   0.09262262]
 [ 0.01254566  0.02317253 -0.04988549 -0.00342947]
 [-0.02093738  0.01336727 -0.03097142  0.01798436]
 [ 0.01856435  0.00323141  0.04157985 -0.04175168]
 [-0.02186538 -0.00669532  0.01176731 -0.0304105 ]
 [-0.04002833 -0.04024273  0.0603158   0.00202133]
 [ 0.06300092 -0.08333338 -0.09999038 -0.02277627]
 [-0.03207715 -0.02979639  0.02165829 -0.02444825]
 [ 0.01658143  0.01263257  0.01796791 -0.01539641]
 [-0.

In [52]:
print('weights[13]:', weights[13])

weights[13]: [ 0.01856435  0.00323141  0.04157985 -0.04175168]


In [53]:
print('weights[4]:', weights[4])

weights[4]: [ 0.07662225  0.07200582  0.07823209 -0.0928142 ]


In [54]:
print('weights[16]:', weights[16])

weights[16]: [ 0.06300092 -0.08333338 -0.09999038 -0.02277627]
